# Input pipeline into Keras

In this notebook, we will look at how to read large datasets, datasets that may not fit into memory, using TensorFlow. We can use the tf.data pipeline to feed data to Keras models that use a TensorFlow backend.

Let's start off with the Python imports that we need.

In [1]:
import os, json, math
import numpy as np
import tensorflow as tf
print(tf.__version__)

2.0.0-dev20190612


## Locating the CSV files

We will start with the CSV files that we wrote out in the [first notebook](../01_explore/taxifare.iypnb) of this sequence. Just so you don't have to run the notebook, we saved a copy in ../data

In [2]:
!ls -l ../data/*.csv

-rw-r--r-- 1 jupyter jupyter 126266 Jun  3 15:48 ../data/taxi-test.csv
-rw-r--r-- 1 jupyter jupyter 593612 Jun  3 15:48 ../data/taxi-train.csv
-rw-r--r-- 1 jupyter jupyter 126833 Jun  3 15:48 ../data/taxi-valid.csv


## Use tf.data to read the CSV files

See the documentation for [make_csv_dataset](https://www.tensorflow.org/api_docs/python/tf/data/experimental/make_csv_dataset).
If you have TFRecords (which is recommended), use [make_batched_features_dataset](https://www.tensorflow.org/api_docs/python/tf/data/experimental/make_batched_features_dataset) instead.

In [3]:
CSV_COLUMNS  = ['fare_amount',  'pickup_datetime',
                'pickup_longitude', 'pickup_latitude', 
                'dropoff_longitude', 'dropoff_latitude', 
                'passenger_count', 'key']
LABEL_COLUMN = 'fare_amount'
DEFAULTS     = [[0.0],['na'],[0.0],[0.0],[0.0],[0.0],[0.0],['na']]

In [4]:
# load the training data
def load_dataset(pattern):
  return tf.data.experimental.make_csv_dataset(pattern, 1, CSV_COLUMNS, DEFAULTS)

tempds = load_dataset('../data/taxi-train*')
print(tempds)

W0612 19:18:50.854974 139700064765696 deprecation.py:323] From /home/jupyter/.local/lib/python3.5/site-packages/tensorflow_core/python/data/experimental/ops/readers.py:498: parallel_interleave (from tensorflow.python.data.experimental.ops.interleave_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use `tf.data.Dataset.interleave(map_func, cycle_length, block_length, num_parallel_calls=tf.data.experimental.AUTOTUNE)` instead. If sloppy execution is desired, use `tf.data.Options.experimental_determinstic`.
W0612 19:18:50.892905 139700064765696 deprecation.py:323] From /home/jupyter/.local/lib/python3.5/site-packages/tensorflow_core/python/data/experimental/ops/readers.py:211: shuffle_and_repeat (from tensorflow.python.data.experimental.ops.shuffle_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use `tf.data.Dataset.shuffle(buffer_size, seed)` followed by `tf.data.Dataset.repeat(count)`. Static tf.data optimizati

<PrefetchDataset shapes: OrderedDict([(fare_amount, (1,)), (pickup_datetime, (1,)), (pickup_longitude, (1,)), (pickup_latitude, (1,)), (dropoff_longitude, (1,)), (dropoff_latitude, (1,)), (passenger_count, (1,)), (key, (1,))]), types: OrderedDict([(fare_amount, tf.float32), (pickup_datetime, tf.string), (pickup_longitude, tf.float32), (pickup_latitude, tf.float32), (dropoff_longitude, tf.float32), (dropoff_latitude, tf.float32), (passenger_count, tf.float32), (key, tf.string)])>


Note that this is a prefetched dataset. If you loop over the dataset, you'll get the rows one-by-one. Let's convert each row into a Python dictionary:

In [5]:
# print a few of the rows
for n, data in enumerate(tempds):
    row_data = {k: v.numpy() for k,v in data.items()}
    print(n, row_data)
    if n > 2:
        break

0 {'fare_amount': array([3.3], dtype=float32), 'dropoff_latitude': array([40.74336], dtype=float32), 'dropoff_longitude': array([-73.9963], dtype=float32), 'passenger_count': array([1.], dtype=float32), 'pickup_datetime': array([b'2011-03-19 03:32:00+00:00'], dtype=object), 'pickup_longitude': array([-73.99346], dtype=float32), 'pickup_latitude': array([40.746967], dtype=float32), 'key': array([b'3736'], dtype=object)}
1 {'fare_amount': array([15.], dtype=float32), 'dropoff_latitude': array([40.763405], dtype=float32), 'dropoff_longitude': array([-73.97676], dtype=float32), 'passenger_count': array([1.], dtype=float32), 'pickup_datetime': array([b'2014-09-16 09:34:40+00:00'], dtype=object), 'pickup_longitude': array([-74.00079], dtype=float32), 'pickup_latitude': array([40.74736], dtype=float32), 'key': array([b'4671'], dtype=object)}
2 {'fare_amount': array([11.], dtype=float32), 'dropoff_latitude': array([40.786392], dtype=float32), 'dropoff_longitude': array([-73.95283], dtype=float

What we really need is a dictionary of features + a label. So, we have to do two things to the above dictionary. (1) remove the unwanted column "key" and (2) keep the label separate from the features.

In [6]:
# get features, label
def features_and_labels(row_data):
    for unwanted_col in ['pickup_datetime', 'key']:
        row_data.pop(unwanted_col)
    label = row_data.pop(LABEL_COLUMN)
    return row_data, label  # features, label

# print a few rows to make it sure works
for n, data in enumerate(tempds):
    row_data = {k: v.numpy() for k,v in data.items()}
    features, label = features_and_labels(row_data)
    print(n, label, features)
    if n > 2:
        break

0 [3.3] {'dropoff_latitude': array([40.74336], dtype=float32), 'dropoff_longitude': array([-73.9963], dtype=float32), 'passenger_count': array([1.], dtype=float32), 'pickup_longitude': array([-73.99346], dtype=float32), 'pickup_latitude': array([40.746967], dtype=float32)}
1 [15.] {'dropoff_latitude': array([40.763405], dtype=float32), 'dropoff_longitude': array([-73.97676], dtype=float32), 'passenger_count': array([1.], dtype=float32), 'pickup_longitude': array([-74.00079], dtype=float32), 'pickup_latitude': array([40.74736], dtype=float32)}
2 [11.] {'dropoff_latitude': array([40.786392], dtype=float32), 'dropoff_longitude': array([-73.95283], dtype=float32), 'passenger_count': array([5.], dtype=float32), 'pickup_longitude': array([-73.96925], dtype=float32), 'pickup_latitude': array([40.763966], dtype=float32)}
3 [5.3] {'dropoff_latitude': array([40.7491], dtype=float32), 'dropoff_longitude': array([-73.983665], dtype=float32), 'passenger_count': array([1.], dtype=float32), 'pickup_l

## Batching

Let's do both (loading, features_label)
in our load_dataset function, and also add batching.

In [7]:
def load_dataset(pattern, batch_size):
  return (
      tf.data.experimental.make_csv_dataset(pattern, batch_size, CSV_COLUMNS, DEFAULTS)
             .map(features_and_labels) # features, label
  )

# try changing the batch size and watch what happens.
tempds = load_dataset('../data/taxi-train*', batch_size=2)
print(list(tempds.take(3))) # truncate and print as a list 

[(OrderedDict([('pickup_longitude', <tf.Tensor: id=295, shape=(2,), dtype=float32, numpy=array([-73.95522, -73.97164], dtype=float32)>), ('pickup_latitude', <tf.Tensor: id=294, shape=(2,), dtype=float32, numpy=array([40.767857, 40.746677], dtype=float32)>), ('dropoff_longitude', <tf.Tensor: id=292, shape=(2,), dtype=float32, numpy=array([-73.98313, -73.96742], dtype=float32)>), ('dropoff_latitude', <tf.Tensor: id=291, shape=(2,), dtype=float32, numpy=array([40.755756, 40.760128], dtype=float32)>), ('passenger_count', <tf.Tensor: id=293, shape=(2,), dtype=float32, numpy=array([1., 6.], dtype=float32)>)]), <tf.Tensor: id=296, shape=(2,), dtype=float32, numpy=array([8.5, 6.5], dtype=float32)>), (OrderedDict([('pickup_longitude', <tf.Tensor: id=301, shape=(2,), dtype=float32, numpy=array([-73.95229 , -73.945946], dtype=float32)>), ('pickup_latitude', <tf.Tensor: id=300, shape=(2,), dtype=float32, numpy=array([40.777195, 40.77777 ], dtype=float32)>), ('dropoff_longitude', <tf.Tensor: id=298

## Shuffling

When training a deep learning model in batches over multiple workers, it is helpful if we shuffle the data. That way, different workers will be working on different parts of the input file at the same time, and so averaging gradients across workers will help. Also, during training, we will need to read the data indefinitely.

In [8]:
def load_dataset(pattern, batch_size=1, mode=tf.estimator.ModeKeys.EVAL):
  dataset = (tf.data.experimental.make_csv_dataset(pattern, batch_size, CSV_COLUMNS, DEFAULTS)
             .map(features_and_labels) # features, label
             .cache())
  if mode == tf.estimator.ModeKeys.TRAIN:
        dataset = dataset.shuffle(1000).repeat()
  dataset = dataset.prefetch(1) # take advantage of multi-threading; 1=AUTOTUNE
  return dataset

tempds = load_dataset('../data/taxi-train*', 2, tf.estimator.ModeKeys.TRAIN)
print(list(tempds.take(1)))
tempds = load_dataset('../data/taxi-valid*', 2, tf.estimator.ModeKeys.EVAL)
print(list(tempds.take(1)))

[(OrderedDict([('pickup_longitude', <tf.Tensor: id=429, shape=(2,), dtype=float32, numpy=array([-73.97753, -73.97575], dtype=float32)>), ('pickup_latitude', <tf.Tensor: id=428, shape=(2,), dtype=float32, numpy=array([40.74272 , 40.758064], dtype=float32)>), ('dropoff_longitude', <tf.Tensor: id=426, shape=(2,), dtype=float32, numpy=array([-73.97834, -73.94462], dtype=float32)>), ('dropoff_latitude', <tf.Tensor: id=425, shape=(2,), dtype=float32, numpy=array([40.721577, 40.780113], dtype=float32)>), ('passenger_count', <tf.Tensor: id=427, shape=(2,), dtype=float32, numpy=array([1., 1.], dtype=float32)>)]), <tf.Tensor: id=430, shape=(2,), dtype=float32, numpy=array([7.7, 8.5], dtype=float32)>)]
[(OrderedDict([('pickup_longitude', <tf.Tensor: id=533, shape=(2,), dtype=float32, numpy=array([-73.87311, -73.98722], dtype=float32)>), ('pickup_latitude', <tf.Tensor: id=532, shape=(2,), dtype=float32, numpy=array([40.774097, 40.7204  ], dtype=float32)>), ('dropoff_longitude', <tf.Tensor: id=530,

In the next notebook, we will build the model using this input pipeline.

Copyright 2019 Google Inc.
Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at
http://www.apache.org/licenses/LICENSE-2.0
Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.